In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# !git clone https://github.com/anandbhoraskar/img2vec.git
# !cd img2vec && git pull origin master

In [0]:
# !unzip /content/drive/My\ Drive/Sem1/10701/data/FID-300.zip 
# !unzip /content/drive/My\ Drive/Sem1/10701/data/CSFID.zip 

In [0]:
import sys
import os
sys.path.insert(1, 'img2vec')
from img2vec_pytorch import Img2Vec
import img2vec_pytorch
import importlib
importlib.reload(img2vec_pytorch)

from PIL import Image
import numpy as np

In [0]:
data_path = "./FID-300/"

In [0]:
from scipy.io import loadmat
label_table = loadmat('FID-300/label_table.mat')
label_table = label_table['label_table']
# print(label_table['label_table'])

In [0]:
references = np.arange(1, 1176)
# print(references)

test_val = np.arange(1, 301)
# print(test_val)

In [0]:
def file_path(i:int, data:str = 'ref'):
  if data == 'ref':
    # print('ref')
    return data_path+"/references/"+"{:05d}".format(i)+".png"
  else:
    # print('test')
    return data_path+"/tracks_cropped/"+"{:05d}".format(i)+".jpg"

In [0]:
print(file_path(1, 'ref'))
print(file_path(1, 'test'))

./FID-300//references/00001.png
./FID-300//tracks_cropped/00001.jpg


In [0]:
ref_img_list = []
for i in references:
  img = Image.open(file_path(i,'ref'))
  img = img.convert("RGB")
  ref_img_list.append(img)

In [0]:
test_img_list = []
for i in test_val:
  img = Image.open(file_path(i,'test'))
  img = img.convert("RGB")
  test_img_list.append(img)

In [0]:
import img2vec_pytorch
import importlib
importlib.reload(img2vec_pytorch)
img2vec = img2vec_pytorch.Img2Vec(cuda=True, layer_output_size=512)

In [0]:
# img2vec.layer_output_size = 2048
# print(img2vec.layer_output_size)

In [0]:
ref_vec_list = []
test_vec_list = []

ref_vec_list = [img2vec.get_vec(img, tensor=True) for img in ref_img_list]
test_vec_list = [img2vec.get_vec(img, tensor=True) for img in test_img_list]

In [0]:
ref_vec_list_np = np.array([np.array(vec) for vec in ref_vec_list])
test_vec_list_np = np.array([np.array(vec) for vec in test_vec_list])

In [0]:
ref_vec_list_np = ref_vec_list_np.reshape(1175, 2048)
test_vec_list_np = test_vec_list_np.reshape(300, 2048)

In [0]:
from sklearn.metrics.pairwise import cosine_similarity
score = cosine_similarity(test_vec_list_np, ref_vec_list_np)

In [0]:
score_sort = (-score).argsort(1)

In [0]:
pos_array = []
for a,b,c in zip(score_sort, score.argmax(1), label_table[:, 1]-1):
  pos_array.append(np.where(a==c)[0][0])
pos_array = np.array(pos_array)
  

In [0]:
t = 5
thresh = t*pos_array.shape[0]/100
acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
print("top {}: {}".format(t, acc))

top 5: 22.666666666666668


In [0]:
t = 10
thresh = t*pos_array.shape[0]/100
acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
print("top {}: {}".format(t, acc))

top 10: 32.666666666666664


In [0]:
def find_scores(img2vec, layer_output_size=512, return_tensor=False):
  if return_tensor:
    ref_vec_list = np.array([img2vec.get_vec(img).mean(1).flatten().numpy() for img in ref_img_list])
    test_vec_list = np.array([img2vec.get_vec(img).mean(1).flatten().numpy() for img in test_img_list])
  else:
    ref_vec_list = np.array([img2vec.get_vec(img) for img in ref_img_list])
    test_vec_list = np.array([img2vec.get_vec(img) for img in test_img_list])

  ref_vec_list = ref_vec_list.reshape(1175, layer_output_size)
  test_vec_list = test_vec_list.reshape(300, layer_output_size)

  # test_vec_list = (test_vec_list-test_vec_list.mean(1)[:, None])/test_vec_list.std(1)[:, None]
  # ref_vec_list = (ref_vec_list-ref_vec_list.mean(1)[:, None])/ref_vec_list.std(1)[:, None]
  score = cosine_similarity(test_vec_list, ref_vec_list)
  score_sort = (-score).argsort(1)

  pos_array = []
  for a,b,c in zip(score_sort, score.argmax(1), label_table[:, 1]-1):
    pos_array.append(np.where(a==c)[0][0])
  pos_array = np.array(pos_array)

  t = 5
  thresh = t*pos_array.shape[0]/100
  acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
  print("top {}: {}".format(t, acc))

  t = 10
  thresh = t*pos_array.shape[0]/100
  acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
  print("top {}: {}".format(t, acc))
  return score

In [0]:
def mcncc(v1, v2):
  den = np.linalg.norm(v1, axis=1)*np.linalg.norm(v2, axis=1) + 1e-12
  num = (v1*v2).sum(axis=1)
  return (num/den).sum()

def find_scores_mcncc(img2vec, layer_output_size=512, return_tensor=False):
  ref_vec_list = np.array([img2vec.get_vec(img).numpy() for img in ref_img_list])
  test_vec_list = np.array([img2vec.get_vec(img).numpy() for img in test_img_list])

  ref_vec_list = ref_vec_list.reshape(ref_vec_list.shape[0], ref_vec_list.shape[2],ref_vec_list.shape[3]*ref_vec_list.shape[4])
  test_vec_list = test_vec_list.reshape(test_vec_list.shape[0], test_vec_list.shape[2], test_vec_list.shape[3]*test_vec_list.shape[4])

  test_vec_list = (test_vec_list-test_vec_list.mean(1)[:, None])/test_vec_list.std(1)[:, None]
  ref_vec_list = (ref_vec_list-ref_vec_list.mean(1)[:, None])/ref_vec_list.std(1)[:, None]
  print("computing score")
  score = np.zeros((test_vec_list.shape[0], ref_vec_list.shape[0]))
  for i, v1 in enumerate(test_vec_list):
    for j, v2 in enumerate(ref_vec_list):
      score[i, j] = mcncc(v1, v2)

  score_sort = (-score).argsort(1)

  pos_array = []
  for a,b,c in zip(score_sort, score.argmax(1), label_table[:, 1]-1):
    pos_array.append(np.where(a==c)[0][0])
  pos_array = np.array(pos_array)

  t = 5
  thresh = t*pos_array.shape[0]/100
  acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
  print("top {}: {}".format(t, acc))

  t = 10
  thresh = t*pos_array.shape[0]/100
  acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
  print("top {}: {}".format(t, acc))
  return score

In [0]:
img2vec = img2vec_pytorch.Img2Vec(cuda=True, layer_output_size=2048, centre_crop=True)
print(img2vec.layer_output_size)
score_layer4 = find_scores(img2vec, 2048)

# Cos sim
# top 5: 22.666666666666668
# top 10: 32.666666666666664

# Normalize:
# top 5: 22.333333333333332
# top 10: 30.333333333333332

2048
top 5: 24.666666666666668
top 10: 35.0


In [0]:
img2vec_layer3 = img2vec_pytorch.Img2Vec(cuda=True, layer='layer3', layer_output_size=1024, centre_crop=True)
score_layer3 = find_scores(img2vec_layer3, 1024)

# Cos
# top 5: 6.0
# top 10: 8.666666666666666

top 5: 22.666666666666668
top 10: 27.666666666666668


In [0]:
img2vec_layer2 = img2vec_pytorch.Img2Vec(cuda=True, layer='layer2', layer_output_size=512, channels=28)
score_layer2 = find_scores(img2vec_layer2, 512)

# Cos
# top 5: 7.0
# top 10: 11.0

top 5: 9.666666666666666
top 10: 13.666666666666666


In [0]:
img2vec_layer1 = img2vec_pytorch.Img2Vec(cuda=True, layer='layer1', layer_output_size=256, channels=56)
score_layer1 = find_scores(img2vec_layer1, 256)

#Cos
# top 5: 5.333333333333333
# top 10: 7.0

top 5: 11.0
top 10: 17.333333333333332


In [0]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer1', layer_output_size=256, 
    channels=56, return_embedding=True)
score_layer2_conv = find_scores(img2vec, 3136, return_tensor=True)

#Cos
# top 5: 7.333333333333333
# top 10: 10.0

top 5: 2.3333333333333335
top 10: 3.6666666666666665


In [0]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer2', layer_output_size=128, 
    channels=28, return_embedding=True)
score_layer2_conv = find_scores(img2vec, 784, return_tensor=True)

# cos
# top 5: 5.0
# top 10: 9.333333333333334

top 5: 7.0
top 10: 9.666666666666666


In [0]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer3', layer_output_size=256, 
    channels=14, return_embedding=True)
score_layer2_conv = find_scores(img2vec, 196, return_tensor=True)

# cos
# top 5: 9.666666666666666
# top 10: 15.666666666666666

top 5: 7.666666666666667
top 10: 12.0


In [0]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer4', layer_output_size=512, 
    channels=7, return_embedding=True)
score_layer2_conv = find_scores(img2vec, 49, return_tensor=True)

# cos
# top 5: 2.3333333333333335
# top 10: 4.0

top 5: 4.333333333333333
top 10: 5.0


In [0]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer1', layer_output_size=64, 
    channels=56, return_embedding=True, centre_crop=True)
score_layer2_conv = find_scores(img2vec, 3136, return_tensor=True)

KeyboardInterrupt: ignored

In [0]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer2', layer_output_size=128, 
    channels=28, return_embedding=True, centre_crop=True)
score_layer2_conv = find_scores(img2vec, 784, return_tensor=True)

In [0]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer3', layer_output_size=256, 
    channels=14, return_embedding=True, centre_crop=True)
score_layer2_conv = find_scores(img2vec, 196, return_tensor=True)

In [0]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer4', layer_output_size=512, 
    channels=7, return_embedding=True, centre_crop=True)
score_layer2_conv = find_scores(img2vec, 49, return_tensor=True)

In [0]:
  ref_vec_list = np.array([img2vec.get_vec(img).numpy() for img in ref_img_list])
  test_vec_list = np.array([img2vec.get_vec(img).numpy() for img in test_img_list])


In [0]:
  ref_vec_list = ref_vec_list.reshape(ref_vec_list.shape[0], ref_vec_list.shape[2],ref_vec_list.shape[3]*ref_vec_list.shape[4])
  test_vec_list = test_vec_list.reshape(test_vec_list.shape[0], test_vec_list.shape[2], test_vec_list.shape[3]*test_vec_list.shape[4])

In [0]:
def mcncc(v1, v2):
  den = np.linalg.norm(v1, axis=1)*np.linalg.norm(v2, axis=1) + 1e-12
  num = (v1*v2).sum(axis=1)
  return (num/den).sum()

In [0]:
  score = np.zeros((test_vec_list.shape[0], ref_vec_list.shape[0]))
  for i, v1 in enumerate(test_vec_list):
    for j, v2 in enumerate(ref_vec_list):
      score[i, j] = mcncc(v1, v2)

In [0]:
  score_sort = (-score).argsort(1)

  pos_array = []
  for a,b,c in zip(score_sort, score.argmax(1), label_table[:, 1]-1):
    pos_array.append(np.where(a==c)[0][0])
  pos_array = np.array(pos_array)

  t = 5
  thresh = t*pos_array.shape[0]/100
  acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
  print("top {}: {}".format(t, acc))

  t = 10
  thresh = t*pos_array.shape[0]/100
  acc = 100*np.sum((pos_array<thresh))/pos_array.shape[0]
  print("top {}: {}".format(t, acc))

In [0]:
v1 = test_vec_list[0]
v2 = ref_vec_list[0]

In [0]:
v1.shape

In [0]:
den = np.linalg.norm(v1, axis=1)*np.linalg.norm(v2, axis=1) + 1e-12
num = (v1*v2).sum(axis=1)
ans = (num/den).sum()

In [0]:
ans

In [0]:
img2vec = img2vec_pytorch.Img2Vec(
    cuda=True, layer='layer3', layer_output_size=256, 
    channels=14, return_embedding=True)
score_layer2_conv = find_scores_mcncc(img2vec, 196, return_tensor=True)

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torchsummary as summary

import sys
import os
sys.path.insert(1, 'img2vec')
from img2vec_pytorch import Img2Vec
import img2vec_pytorch
import importlib
importlib.reload(img2vec_pytorch)

from PIL import Image
import numpy as np

import img2vec_pytorch
import importlib
importlib.reload(img2vec_pytorch)
img2vec = img2vec_pytorch.Img2Vec(cuda=True, layer_output_size=2048)

In [0]:
model = models.resnet50(pretrained=True)

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# summary.summary(model, (3, 224, 224))

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [0]:
img

In [0]:
image = img2vec.normalize(img2vec.to_tensor(img2vec.scaler(img))).unsqueeze(0).to(img2vec.device)
img2vec.model(image)

In [0]:
img2vec.model

In [0]:
image = img2vec.normalize(img2vec.to_tensor(img2vec.scaler(img))).unsqueeze(0).to(img2vec.device)
global my_embedding
# my_embedding = torch.zeros(1, img2vec.layer_output_size, img2vec.channels, img2vec.channels)


def copy_data(m, i, o):
    global s
    global my_embedding
    s = o.data.clone()
    # print("my_embedding in1= {}".format(my_embedding))
    # my_embedding = o.data.clone()
    # my_embedding.copy_(o.data)
    my_embedding = o.data.clone()
    print("my_embedding in= {}".format(my_embedding))
    # .copy_(o.data)

h = img2vec.extraction_layer.register_forward_hook(copy_data)
h_x = img2vec.model(image)
h.remove()
print("my_embedding out = {}".format(my_embedding))

# if img2vec.return_embedding:
#     return my_embedding

my_embedding = F.adaptive_avg_pool2d(my_embedding, (1, 1))

In [0]:
img2vec.get_vec